# **NLP HW 3**

## **Дарья Родионова БКЛ182**

Работа сделана в Google Colab.

### **Importing packages**

Для начала импортируем все нужные модули и библиотеки.

In [1]:
!pip install nltk

In [2]:
!pip install spacy

In [3]:
!pip  install pyLDAvis

In [4]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
!python3 -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [6]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import spacy
import nltk

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

### **Preprocessing**

In [7]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [8]:
data = df.content.values.tolist()

data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

data = [re.sub('\s+', ' ', sent) for sent in data]

data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [9]:
data[1]

'From: (Guy Kuo) Subject: SI Clock Poll - Final Call Summary: Final call for SI clock reports Keywords: SI,acceleration,clock,upgrade Article-I.D.: shelley.1qvfo9INNc3s Organization: University of Washington Lines: 11 NNTP-Posting-Host: carson.u.washington.edu A fair number of brave souls who upgraded their SI clock oscillator have shared their experiences for this poll. Please send a brief message detailing your experiences with the procedure. Top speed attained, CPU rated speed, add on cards and adapters, heat sinks, hour of usage per day, floppy disk functionality with 800 and 1.4 m floppies are especially requested. I will be summarizing in the next two days, so please add to the network knowledge base if you have done the clock upgrade and havent answered this poll. Thanks. Guy Kuo '

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [11]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [12]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(' '.join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words_nostops)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [14]:
id2word = corpora.Dictionary(data_lemmatized)

texts = data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]

### **Mallet**

In [15]:
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


Скачиваем Java, чтобы Mallet нормально работал.

In [16]:
import os  
def install_java():
    !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null   
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     
    !java -version       
install_java()

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


In [17]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

--2020-10-19 14:01:32--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip.4’

mallet-2.0.8.zip.4  100%[===================>]  15.43M  10.9MB/s    in 1.4s    

2020-10-19 14:01:34 (10.9 MB/s) - ‘mallet-2.0.8.zip.4’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [18]:
import os
from gensim.models.wrappers import LdaMallet

os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet'

Функция находит модель с оптимальным количеством топиков. 

In [19]:
def number_of_topics(mallet_path, corpus, id2word, data_lemmatized, start, stop, step):
    best_number = 0
    best_model = 0
    for i in tqdm(range(start, stop+1, step)):
        ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=i, id2word=id2word)
        coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
        coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    if coherence_ldamallet > best_model:
        best_model = coherence_ldamallet
        best_number = i
    return best_number, best_model

In [20]:
best_number, best_model = number_of_topics(mallet_path, corpus, id2word, data_lemmatized, 1, 35, 10)
print('\nNumber of topics:', best_number)
print('\nCoherence:', best_model)

100%|██████████| 4/4 [09:41<00:00, 145.40s/it]


Number of topics: 31

Coherence: 0.543512710735776


Оптимальное число получилось 31, поэтому запускаем модель именно с этим числом.

In [21]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=best_number, id2word=id2word)
gensimmodel = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)

In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(gensimmodel, corpus, id2word)
vis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
1      0.212202 -0.137647       1        1  4.109246
12    -0.160407 -0.013105       2        1  4.073101
19    -0.190197 -0.046473       3        1  4.059310
16    -0.156517 -0.067138       4        1  3.977852
24    -0.005465  0.033536       5        1  3.746450
23    -0.122890 -0.060807       6        1  3.599114
6     -0.133438 -0.082467       7        1  3.596078
17     0.041688 -0.171717       8        1  3.590351
7      0.151393 -0.166042       9        1  3.588241
3     -0.187779 -0.078428      10        1  3.532581
11    -0.140972 -0.083275      11        1  3.473774
13    -0.178565 -0.054733      12        1  3.413701
10     0.002245 -0.045756      13        1  3.330816
22     0.098159 -0.063600      14        1  3.303623
26    -0.136683  0.134536      15        1  3.225114
15    -0.081815  0.159101      16        1  3.204167
18    -0.037354 -0.040477      17        1  3.167455
25     0.074659 -0.108206      18        1  3.135027
30     0.067041 -0.062490      19        1  3.131195
8      0.059027 -0.116929      20        1  3.114876
14     0.074748  0.055032      21        1  3.067723
28    -0.177177  0.061766      22        1  3.021009
2     -0.120303  0.120616      23        1  2.914623
5      0.096257 -0.055071      24        1  2.907792
21     0.257206 -0.064023      25        1  2.896016
20     0.042366  0.129376      26        1  2.893232
27     0.234430 -0.068819      27        1  2.705798
29     0.092788  0.078386      28        1  2.624282
4      0.075935  0.328209      29        1  2.452310
0      0.139297  0.305256      30        1  2.111312
9      0.110119  0.181389      31        1  2.033831, topic_info=         Term          Freq         Total Category  logprob  loglift
19       line  14562.000000  14562.000000  Default  30.0000  30.0000
197     write  12161.000000  12161.000000  Default  29.0000  29.0000
3685       ax   8024.000000   8024.000000  Default  28.0000  28.0000
177   article   7723.000000   7723.000000  Default  27.0000  27.0000
1780      max   6466.000000   6466.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
374      part    129.262200   2250.797085  Topic31  -5.1799   1.0381
495       end     71.434374   1753.668383  Topic31  -5.7729   0.6946
5148   senior     28.913913    102.437900  Topic31  -6.6774   2.6303
1382      cut     39.118824    662.245657  Topic31  -6.3751   1.0662
60    keyword     28.913913    556.548914  Topic31  -6.6774   0.9378

[3074 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
6945      30  1.021334   aamir_qazi
18219     10  1.000791       abduct
3907      20  0.996233        abort
4863       6  0.061558     abortion
4863       7  0.007695     abortion
...      ...       ...          ...
22582      4  1.198134  yugoslavian
3656      24  1.016397         zinc
7614      10  1.000791      zionism
7615      10  0.991260      zionist
18695     14  0.781540        zurlo

[5939 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 13, 20, 17, 25, 24, 7, 18, 8, 4, 12, 14, 11, 23, 27, 16, 19, 26, 31, 9, 15, 29, 3, 6, 22, 21, 28, 30, 5, 1, 10])

In [23]:
topics = ldamallet.show_topics(formatted=False, num_topics=31)

Функция находит главную тему в каждом тексте. Для этого мы находим номер темы, который имеет самый большой вес.

In [24]:
topics_dict = {}

for i, text in enumerate(ldamallet[corpus]):
    text = sorted(text, key=lambda x: x[1], reverse=True)
    main_topic = text[0][0] # dominant topic
    if main_topic not in topics_dict:
        topics_dict[main_topic] = []
    topics_dict[main_topic].append(i)

### **Tf-Idf**

Cчитаем td-idf внутри текстов с одним широким топиком. Для каждого текста берём список tf_idf его слов. Пять слов с самым высоким tf_idf склеиваем в одну строку, а потом созадём датафрейм с соответствующей информацией. 

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [47]:
def top_df_words(texts, data, topics_dict):
  df = pd.DataFrame(columns=['texts', 'topic', 'text_id', 'top_five'])
  for topic in topics_dict:
    words = [' '.join(texts[w]) for w in topics_dict[topic]] 
    X = vectorizer.fit_transform(words)
    feature_names = vectorizer.get_feature_names()
    vectors = X.todense().tolist()
    result = zip(vectors, topics_dict[topic])
    for v, num in result: # v = list(tfidf), num = text_id
      keywords = ' '.join([feature_names[w[0]] for w in sorted([(i, j) for i, j in enumerate(v)], key=lambda x: (x[1]), reverse=True)[:5]]) # sorting values by 'tfidf' in descending order
      df = df.append({'texts': data[num], 'topic': topic, 'text_id': num, 'top_five': keywords}, ignore_index=True)
  return df

In [48]:
tf_idf = top_df_words(texts, data, topics_dict)

In [51]:
tf_idf

,texts,topic,text_id,top_five
0,From: (wheres my thing) Subject: WHAT car is t...,14,0,car door bricklin lerxst tellme
1,From: (Irwin Arnstein) Subject: Re: Recommenda...,14,10,ducati mate bronze computrac_inc gts
2,From: (Kevin Parker) Subject: Insurance Rates ...,14,17,year insurance rate age car
3,From: (Jim Frost) Subject: Re: Is car saftey i...,14,29,priority collision safety important mph
4,From: (eliot) Subject: Re: Improvements in Aut...,14,56,gear throttle mode automatic assumption
...,...,...,...,...
11309,Organization: City University of New York From...,6,11141,admit incredible liechenstein nowadays also
11310,From: (Steinn Sigurdsson) Subject: Re: New pla...,6,11172,object revolution new lick_observatory slide
11311,From: (Keith Allan Schneider) Subject: Re: Is ...,6,11194,would okay longer religious organization
11312,From: (Walter Rolandi) Subject: Re: Will Italy...,6,11273,accountable efficient bring party system


### **Coherence Score**

Coherence Score это степень семантической близости между высоко оценёнными словами в теме. С помощью этой метрики можно понять, насколько логичен результат нашей модели (ключевые слова в теме). 

Расчёт с помощью меры c_v состоит из следующих этапов:
1. Деление данных на пары слов.
2. Вычесление их вероятности.
3. Вычесление confirmation measure, которая считает согласованность между парами. 
4. Высчитывание общего coherence score путём объединения individual confirmation measure.